In [1]:
import pandas as pd
from tqdm import tqdm
from collections import defaultdict
import matplotlib.pyplot as plt
import os
import statistics

In [36]:
df  = pd.read_excel(r"../data/test3.xlsx", sheet_name="Обрывы_акт.мощность")
df.head()

,mst,skv,p,dt
0,Александровская,2513,8.80000019,2024-02-06 04:03:25
1,Александровская,2513,4.20000029,2024-02-06 04:04:28
2,Александровская,2513,3,2024-02-06 04:06:26
3,Александровская,2513,5.5,2024-02-06 04:07:25
4,Александровская,2513,9.5,2024-02-06 04:08:27


In [37]:
df.shape

(110009, 4)

In [38]:
df.drop(["Мест-е"], axis=1, inplace=True, errors="ignore")
df.drop(["Площадь"], axis=1, inplace=True, errors="ignore")
df.drop(["mst"], axis=1, inplace=True, errors="ignore")
df

,skv,p,dt
0,2513,8.80000019,2024-02-06 04:03:25
1,2513,4.20000029,2024-02-06 04:04:28
2,2513,3,2024-02-06 04:06:26
3,2513,5.5,2024-02-06 04:07:25
4,2513,9.5,2024-02-06 04:08:27
...,...,...,...
110004,86,4.0999999,2024-02-15 02:00:04.000
110005,86,8.19999981,2024-02-15 02:02:04.000
110006,86,9.40000057,2024-02-15 02:03:03
110007,86,8.10000038,2024-02-15 02:03:33


In [6]:
df.info();

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110009 entries, 0 to 110008
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   skv     110008 non-null  object
 1   p       110009 non-null  object
 2   dt      110008 non-null  object
dtypes: object(3)
memory usage: 2.5+ MB


In [40]:
df["p"] = df.apply(lambda x: str(x.p).replace(",", "."), axis=1)
df

,skv,p,dt
0,2513,8.80000019,2024-02-06 04:03:25
1,2513,4.20000029,2024-02-06 04:04:28
2,2513,3,2024-02-06 04:06:26
3,2513,5.5,2024-02-06 04:07:25
4,2513,9.5,2024-02-06 04:08:27
...,...,...,...
110004,86,4.0999999,2024-02-15 02:00:04.000
110005,86,8.19999981,2024-02-15 02:02:04.000
110006,86,9.40000057,2024-02-15 02:03:03
110007,86,8.10000038,2024-02-15 02:03:33


In [41]:
def get_num(x):
    try:
        if isinstance(float(x), float):
            return round(float(x),2)
    except:
        return None

In [42]:
df["p"] = df["p"].apply(lambda x: get_num(x))

In [43]:
df["p"].value_counts()

p
0.00     17616
1.40      2630
1.20      2408
1.30      2264
1.10      1652
         ...  
20.82        1
15.27        1
19.53        1
14.28        1
18.82        1
Name: count, Length: 2483, dtype: int64

In [45]:
df = df.query("p >=-100 and p<=100")

In [46]:
try:
    print(df['Значение'].min(), df['Значение'].max(), df['Значение'].mean())
except:

    print(df['p'].min(), df['p'].max(), df['p'].mean())

-70.3 47.1 3.9799290948847674


In [47]:
df

,skv,p,dt
0,2513,8.8,2024-02-06 04:03:25
1,2513,4.2,2024-02-06 04:04:28
2,2513,3.0,2024-02-06 04:06:26
3,2513,5.5,2024-02-06 04:07:25
4,2513,9.5,2024-02-06 04:08:27
...,...,...,...
110004,86,4.1,2024-02-15 02:00:04.000
110005,86,8.2,2024-02-15 02:02:04.000
110006,86,9.4,2024-02-15 02:03:03
110007,86,8.1,2024-02-15 02:03:33


In [48]:
dicter = defaultdict(list)
for index, row in df.iterrows():
    dicter[row["skv"]].append(round(row["p"],2))

In [43]:
dicter.keys()

dict_keys([2513, 751, '11088Г', 335, 384, 6284, 880, 120, 443, 654, '10КНД', 2066, 107, 43, 10008, '129НБР', 191, '30УЗБ', 3207, 3825, 4935, 958, '81СВА', 1949, 2040, 808, 1322, '4373г', 1260, 2176, 2806, 2837, 3392, 479, 702, 1401, '207г', 86, nan])

In [45]:
#plt.figure(figsize=(1.28, 1.28))

for k, v in tqdm(dicter.items()):
    count = 0
    os.makedirs(f'../images//Скв_{k}', exist_ok=True)
    #if k != 1101:continue
    while 12 * (count+1)<=len(v):
        dots = v[count*12:(count+1)*12]
        if len(dots) !=12:print(len(dots))
        if 0.65 < statistics.pstdev(dots) < 0.95:print(k, count+1)
        if statistics.pstdev(dots)<0.65:normalized_data = list(map(lambda x: 0.3, dots))
        else: normalized_data = [(value - min(dots)) / (max(dots) - min(dots)) for value in dots]
        #print(count, normalized_data, dots)
        count+=1
        #plt.axis([0, 12, 0, 1])
        plt.axis("off")
        plt.plot(range(12), normalized_data, linewidth=1, color="black")
        #plt.plot(range(12), normalized_data, "ro")
        for index, y in enumerate(normalized_data):
            pass
            #plt.text(index,y,dots[index])
        plt.savefig(os.path.join(f'../images/Скв_{k}', f'scv_{k}_{count}'), pad_inches=0)

        plt.clf()

 23%|██▎       | 9/39 [00:04<00:16,  1.77it/s]

10КНД 19


 46%|████▌     | 18/39 [00:08<00:09,  2.31it/s]

3207 7
3207 23


 67%|██████▋   | 26/39 [00:12<00:06,  1.92it/s]

...

2806 2
2806 7
2806 8


 79%|███████▉  | 31/39 [00:16<00:06,  1.27it/s]

2806 17
2806 19


100%|██████████| 39/39 [00:19<00:00,  2.02it/s]


<Figure size 640x480 with 0 Axes>

In [33]:
mas1 = [2,3,2,2,2,3,1,2,2,1.3]
mas2 = [1,1,16,1,1,8, 4,6,16,9,2,8]
#mas3 = [2.0, 2.1, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0]
mas3 = [-1, 1, -9, 0, 0,1, 1, -1,]
mas4 = [10,10,10,10,10,10,12,12,12,12,12,12]
mas5 = [16,2,16,2,16,2,16,2,16,2,16,2]
mas6 = [98,96,98,96,98,96,98,96,98,96,98,96]
mas7 = [3.79999995,
4.20000029,
5.20000029,
2.9000001,
3.70000005,
5.0999999,
4.80000019,
5.5999999,
5.5,
4.9000001,
5.70000029,
5.30000019,
]

In [34]:
print(f'mas1. mean = {statistics.mean(mas1)}, median = {statistics.median(mas1)}, sum = {sum(mas1)}, std = {statistics.variance(mas1)}, psdev = {statistics.pstdev(mas1)}, pvariancee = {statistics.pvariance(mas1)}')
print(f'mas2. mean = {statistics.mean(mas2)}, median = {statistics.median(mas2)}, sum = {sum(mas2)}, std = {statistics.variance(mas2)}, psdev = {statistics.pstdev(mas2)}, pvariancee = {statistics.pvariance(mas2)}')
print(f'mas4. mean = {statistics.mean(mas4)}, median = {statistics.median(mas4)}, sum = {sum(mas4)}, std = {statistics.variance(mas4)}, psdev = {statistics.pstdev(mas4)}, pvariancee = {statistics.pvariance(mas4)}')
print(f'mas3. mean = {statistics.mean(mas3)}, median = {statistics.median(mas3)}, sum = {sum(mas3)}, std = {statistics.variance(mas3)}, psdev = {statistics.pstdev(mas3)}, pvariancee = {statistics.pvariance(mas3)}')
print(f'mas5. mean = {statistics.mean(mas5)}, median = {statistics.median(mas5)}, sum = {sum(mas5)}, std = {statistics.variance(mas5)}, psdev = {statistics.pstdev(mas5)}, pvariancee = {statistics.pvariance(mas5)}')
print(f'mas6. mean = {statistics.mean(mas6)}, median = {statistics.median(mas6)}, sum = {sum(mas5)}, std = {statistics.variance(mas6)}, psdev = {statistics.pstdev(mas6)}, pvariancee = {statistics.pvariance(mas6)}')
print(f'mas7. mean = {statistics.mean(mas7)}, median = {statistics.median(mas7)}, sum = {sum(mas7)}, std = {statistics.variance(mas7)}, psdev = {statistics.pstdev(mas7)}, pvariancee = {statistics.pvariance(mas7)}')

mas1. mean = 2.03, median = 2.0, sum = 20.3, std = 0.3867777777777778, psdev = 0.59, pvariancee = 0.3481
mas2. mean = 6.083333333333333, median = 5.0, sum = 73, std = 30.62878787878788, psdev = 5.29871577732651, pvariancee = 28.07638888888889
mas4. mean = 11, median = 11.0, sum = 132, std = 1.0909090909090908, psdev = 1.0, pvariancee = 1
mas3. mean = -1, median = 0.0, sum = -8, std = 11.142857142857142, psdev = 3.122498999199199, pvariancee = 9.75
mas5. mean = 9, median = 9.0, sum = 108, std = 53.45454545454545, psdev = 7.0, pvariancee = 49
mas6. mean = 97, median = 97.0, sum = 108, std = 1.0909090909090908, psdev = 1.0, pvariancee = 1
mas7. mean = 4.725000104166667, median = 5.0, sum = 56.70000125, std = 0.7784091085000219, psdev = 0.8447139650743835, pvariancee = 0.7135416827916867


In [150]:
len(range(12))

12

In [10]:
df

,Скважина,p,Дата-время
0,2513,8.400001,2024-02-06 04:03:41
1,2513,2.900000,2024-02-06 04:04:42
2,2513,9.900001,2024-02-06 04:06:41
3,2513,1.600000,2024-02-06 04:07:40
4,2513,1.800000,2024-02-06 04:08:41
...,...,...,...
110003,86,12.000000,2024-02-15 01:58:04.040
110004,86,4.100000,2024-02-15 02:00:04.000
110005,86,8.200000,2024-02-15 02:02:04.000
110006,86,9.400001,2024-02-15 02:03:03


In [12]:
import plotly.express as px

for i in list(df['Скважина'].unique()):
    print(i)
    max_I = max(df[df['Скважина']==i]['p'])
    fig = px.line(df[df['Скважина']==i], x='Дата-время', y='p')
    fig.update_layout(showlegend=False)
    fig.update_xaxes(visible=False)
    fig.update_yaxes(visible=False, range=[-max_I, max_I])
    fig.show(width=200)

2513


ValueError: Value of 'x' is not the name of a column in 'data_frame'. Expected one of ['Скважина', 'p', 'Дата-время'] but received: Date

In [1]:
!pip install plotly

   ---------------------------------------- 0.0/15.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/15.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/15.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/15.7 MB 445.2 kB/s eta 0:00:36
   ---------------------------------------- 0.0/15.7 MB 445.2 kB/s eta 0:00:36
   ---------------------------------------- 0.1/15.7 MB 525.1 kB/s eta 0:00:30
   ---------------------------------------- 0.1/15.7 MB 525.1 kB/s eta 0:00:30
   ---------------------------------------- 0.1/15.7 MB 450.6 kB/s eta 0:00:35
   ---------------------------------------- 0.2/15.7 MB 459.5 kB/s eta 0:00:34
   ---------------------------------------- 0.2/15.7 MB 525.1 kB/s eta 0:00:30
   ---------------------------------------- 0.2/15.7 MB 525.1 kB/s eta 0:00:30
    --------------------------------------- 0.2/15.7 MB 492.1 kB/s eta 0:00:32
    --------------------------------------- 0.3/15.7 MB 614.4 kB/s eta 0:00:26
